# Data Preprocess

## 0. imports

In [1]:
%load_ext lab_black

In [2]:
import os
import gc
import json
import math
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from eunjeon import Mecab

In [3]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader

In [4]:
import warnings

warnings.filterwarnings(action="ignore")

In [5]:
pl.seed_everything(seed=42)

Global seed set to 42


42

## 1. Data Load

```
1. train.json : 학습용 데이터셋
 ├ id : 회의록 id
 ├ region : 회의 지역
 ├ num_agenda : 안건 수
 │ ├ AGENDA_1 : 안건1
 │ ├ AGENDA_2 : 안건2
 │ ├ AGENDA_3 : 안건3
 │ └ ...
 └ label : 안건별 요약문
   ├ AGENDA_1 : 안건1의 요약문
   │ ├ evidence : 요약 근거
   │ └ summary : 요약문(정답)
   ├ AGENDA_2 : 안건2의 요약문
   │ ├ evidence : 요약 근거
   │ └ summary : 요약문(정답)
   ├ AGENDA_3 : 안건3의 요약문
   │ ├ evidence : 요약 근거
   │ └ summary : 요약문(정답)
   └ ...

2. test.json : 평가용 데이터셋
 ├ id : 회의록 id
 ├ region : 회의 지역
 ├ num_agenda : 안건 수
 └ context : 회의내용
   ├ AGENDA_1 : 안건1
   ├ AGENDA_2 : 안건2
   ├ AGENDA_3 : 안건3
   └ ...

3. sample_submission.csv : 제출용 양식
 - uid : test.json의 안건별 고유 id ('id2000-AGENDA_1 : id2000회의의 안건1)
 - summary : 안건의 요약문
```

In [6]:
DIR = "../data"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [7]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [10]:
train = pd.DataFrame(
    columns=["uid", "num_agenda", "title", "region", "context", "evidence", "summary"]
)
uid = 1000
for data in TRAIN_DATA:
    for agenda in data["context"].keys():
        context = ""
        for line in data["context"][agenda]:
            context += data["context"][agenda][line]
            context += " "
        train.loc[uid, "uid"] = uid
        train.loc[uid, "num_agenda"] = data["num_agenda"]
        train.loc[uid, "title"] = data["title"]
        train.loc[uid, "region"] = data["region"]
        train.loc[uid, "context"] = context[:-1]
        train.loc[uid, "summary"] = data["label"][agenda]["summary"]
        uid += 1
test = pd.DataFrame(columns=["uid", "num_agenda", "title", "region", "context"])
uid = 2000
for data in TEST_DATA:
    for agenda in data["context"].keys():
        context = ""
        for line in data["context"][agenda]:
            context += data["context"][agenda][line]
            context += " "
        test.loc[uid, "uid"] = uid
        test.loc[uid, "num_agenda"] = data["num_agenda"]
        test.loc[uid, "title"] = data["title"]
        test.loc[uid, "region"] = data["region"]
        test.loc[uid, "context"] = context[:-1]
        uid += 1

In [11]:
train["total"] = train.title + " " + train.region + " " + train.context
test["total"] = test.title + " " + test.region + " " + test.context

In [12]:
train.head()

,uid,num_agenda,title,region,context,evidence,summary,total
1000,1000,4,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,NaN,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,1001,4,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,NaN,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,1002,4,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,NaN,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,1003,4,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,NaN,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,1004,5,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,NaN,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [13]:
test.head()

,uid,num_agenda,title,region,context,total
2000,2000,6,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...,제235회 본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001,2001,6,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군..."
2002,2002,6,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록..."
2003,2003,6,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결..."
2004,2004,6,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분..."


In [14]:
test.shape

(506, 6)

In [15]:
train.to_csv("../data/train.csv", index=False)
test.to_csv("../data/test.csv", index=False)